In [4]:
import pandas as pd
import numpy as np

In [5]:
# Name the file variable
file = 'EPA_Walkability_Data.csv'

# Load the CSV into a Dataframe
epa_df = pd.read_csv(file)
epa_df

FileNotFoundError: [Errno 2] No such file or directory: 'EPA_Walkability_Data.csv'

In [6]:
list(epa_df)

NameError: name 'epa_df' is not defined

In [ ]:
epa_df.count()

In [ ]:
# Selecting Necessary Columns and Renaming Columns
epa_columns_df = epa_df[['GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'NatWalkInd', 'CountHU', 'TotPop', 'Pct_AO0', 'Pct_AO1', 'Pct_AO2p', 'R_PCTLOWWAGE', 'D1A', 'D1B', 'D1C', 'D3APO', 'D4D']]
epa_columns_df

In [ ]:
epa_renamed_df = epa_columns_df.rename(
    columns={'GEOID10':'2010 Census ID',
            'GEOID20': '2018 Census ID',
            'STATEFP': 'State FIPS',
            'COUNTYFP': 'County FIPS',
            'TRACTCE': 'Tract FIPS',
            'NatWalkInd': 'Walkability',
            'CountHU': 'Number of Households',
            'TotPop': 'Total Population',
            'Pct_AO0': '% 0 Vehicle HH',
            'Pct_AO1': '% 1 Vehicle HH',
            'Pct_AO2p': '% 2+ Vehicle HH',
            'R_PCTLOWWAGE': '% Low Wage Workers',
            'D1A': 'Residential Density',
            'D1B': 'Population Density',
            'D1C': 'Employment Density',
            'D3APO': 'Pedestrian Network Density',
            'D4D': 'Transit Frequency'})

epa_renamed_df

In [ ]:
# Check for missing values
epa_renamed_df.isnull().sum()

In [ ]:
# Will need to find new dataset on housing density so as not to skew results.
# Drop two columns with significant null values
epa_renamed_df.drop(['Number of Households', 'Residential Density'], axis=1, inplace=True)

epa_renamed_df.isnull().sum()

In [ ]:
# Drop null rows
epa_clean_df = epa_renamed_df.dropna(how='any')

In [ ]:
# Verify that null values are dropped
epa_clean_df.isnull().sum()

In [ ]:
# Drop Census Tract IDs
epa_columns_df = epa_clean_df[['2010 Census ID', 'Tract FIPS','State FIPS', 'County FIPS', 'Walkability', 'Total Population', '% 0 Vehicle HH', '% 1 Vehicle HH', '% 2+ Vehicle HH', 'Pedestrian Network Density', 'Transit Frequency']]
epa_columns_df.set_index(['State FIPS'], inplace=True)
epa_columns_df

In [ ]:
epa_columns_df['2010 Census ID'] = epa_columns_df['2010 Census ID'].div(10000000)
epa_columns_df                        

In [ ]:
epa_columns_df['2010 Census ID'] = epa_columns_df['2010 Census ID'].astype(int)
epa_columns_df

In [ ]:
#Open the state FIPS code CSV
state_code_file = 'StatesFIPSCodes.csv'
state_df = pd.read_csv(state_code_file)
state_df.head()

In [ ]:
state_columns_df = state_df[['STATE_FIPS', 'STATE_NAME']]
state_columns_df.head()

In [ ]:
state_renamed_df = state_columns_df.rename(
    columns={'STATE_FIPS':'State FIPS',
            'STATE_NAME': 'State'})
state_renamed_df.set_index(['State FIPS'], inplace=True)
state_renamed_df.head()
            

In [ ]:
epa_states = epa_columns_df.merge(state_renamed_df, how='inner', on='State FIPS')
epa_states.set_index(['State'], inplace=True)
epa_states

In [ ]:
clean_epa_df = epa_states.rename(
    columns={'2010 Census ID':'Census Tract Number'})
clean_epa_df
            

In [ ]:
# Name the file variable
food_file = 'food_access_clean.csv'

# Load the CSV into a Dataframe
food_df = pd.read_csv(food_file, names=['Census Tract Number', 'State', 'County', 'LILA 1 and 10', 'LILA 0.5 and 10', 'LILA 1 and 20', 'LILA Vehicle'])
food_df

In [ ]:
food_df['Census Tract Number'] = food_df['Census Tract Number'].astype(str)
food_df['Census Tract Number'] = food_df['Census Tract Number'].str[5:].astype(int)
food_df

In [ ]:
walk_food_df = food_df.merge(clean_epa_df, how='inner', on='Census Tract Number')
walk_food_df.set_index(['Census Tract Number'], inplace=True)
walk_food_df

In [ ]:
sorted_df = walk_food_df.sort_index()
sorted_df

In [ ]:
cleaned_location_df = sorted_df[['State', 'County', 'Total Population', 'Walkability', 'Pedestrian Network Density', 'Transit Frequency', '% 0 Vehicle HH', '% 1 Vehicle HH', '% 2+ Vehicle HH', 'LILA 1 and 10', 'LILA 0.5 and 10', 'LILA 1 and 20', 'LILA Vehicle']]
cleaned_location_df

In [ ]:
cleaned_location_df.dtypes

In [ ]:
# Export to a CSV
cleaned_location_df.to_csv('location_data.csv', header=True)